# Inter-domain comparison of signal generator

- Assert example simulations consisting of a slack, a pi-line and a PQ load in either EMT, DP and SP domain
- Simulate with a time step of 1ms over a time frame of 1s

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import re

In [ ]:
 %%bash
TOP=${TOP:-$(git rev-parse --show-toplevel)}
PATH=${TOP}/build/Examples/Cxx

EMT_DP_SP_Slack_PiLine_PQLoad_FM

## Linear frequency ramp

In [ ]:
modelName = 'EMT_DP_SP_Slack_PiLine_PQLoad_FM_linearRamp_EMT'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim_EMT = read_timeseries_csv(dpsim_result_file)

In [ ]:
modelName = 'EMT_DP_SP_Slack_PiLine_PQLoad_FM_linearRamp_DP'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim_DP = read_timeseries_csv(dpsim_result_file)
ts_dpsim_DP_shifted = ts.frequency_shift_list(ts_dpsim_DP, 50)

In [ ]:
modelName = 'EMT_DP_SP_Slack_PiLine_PQLoad_FM_linearRamp_SP'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim_SP = read_timeseries_csv(dpsim_result_file)

### Plot frequency

In [ ]:
plt.figure()

# EMT
plt.plot(ts_dpsim_EMT['f_src'].time, ts_dpsim_EMT['f_src'].values, label='EMT')
    
# DP
plt.plot(ts_dpsim_DP['f_src'].time, ts_dpsim_DP['f_src'].values+50, label='DP', linestyle='--')

# SP
plt.plot(ts_dpsim_DP['f_src'].time, ts_dpsim_DP['f_src'].values+50, label='SP', linestyle=':')
    
plt.legend()
plt.show()

### Plot voltage

In [ ]:
plt.figure()

# EMT
var_names = ['v1_0']
for var_name in var_names: 
    plt.plot(ts_dpsim_EMT[var_name].time, np.sqrt(3/2)*ts_dpsim_EMT[var_name].values, label=var_name)
    
# DP
var_names = ['v1_shift']
for var_name in var_names: 
    plt.plot(ts_dpsim_DP_shifted[var_name].time, ts_dpsim_DP_shifted[var_name].values, label=var_name, linestyle=':')
    
plt.legend()
plt.show()

### Assertion EMT vs. DP

In [ ]:
compare_errors_abs = []
for name in [('v1_0', 'v1_shift'), ('i12_0', 'i12_shift')]:
    compare_errors_abs.append(np.absolute(np.sqrt(3/2)*ts_dpsim_EMT[name[0]].values - ts_dpsim_DP_shifted[name[1]].values).max())
    print(name[0]+' vs. '+name[1] + ' (abs): ' + str(compare_errors_abs[-1]))

assert compare_errors_abs[0] < 1e-3 # voltage
assert compare_errors_abs[1] < 0.9    # current

### Assertion DP vs. SP

In [ ]:
compare_errors_abs = []
for name in ['v1', 'i12']:
    compare_errors_abs.append(np.absolute(ts_dpsim_DP[name].values - ts_dpsim_SP[name].values).max())
    print(name+' vs. '+name + ' (abs): ' + str(compare_errors_abs[-1]))

assert compare_errors_abs[0] < 1e-3 # voltage
assert compare_errors_abs[1] < 0.2    # current

## Cosine frequency modulation

In [ ]:
modelName = 'EMT_DP_SP_Slack_PiLine_PQLoad_FM_cosineFM_EMT'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim_EMT = read_timeseries_csv(dpsim_result_file)

In [ ]:
modelName = 'EMT_DP_SP_Slack_PiLine_PQLoad_FM_cosineFM_DP'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim_DP = read_timeseries_csv(dpsim_result_file)
ts_dpsim_DP_shifted = ts.frequency_shift_list(ts_dpsim_DP, 50)

In [ ]:
modelName = 'EMT_DP_SP_Slack_PiLine_PQLoad_FM_cosineFM_SP'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim_SP = read_timeseries_csv(dpsim_result_file)

### Plot frequency

In [ ]:
plt.figure()

# EMT
plt.plot(ts_dpsim_EMT['f_src'].time, ts_dpsim_EMT['f_src'].values, label='EMT')
    
# DP
plt.plot(ts_dpsim_DP['f_src'].time, ts_dpsim_DP['f_src'].values+50, label='DP', linestyle='--')

# SP
plt.plot(ts_dpsim_DP['f_src'].time, ts_dpsim_DP['f_src'].values+50, label='SP', linestyle=':')
    
plt.legend()
plt.show()

### Assertion EMT vs. DP

In [ ]:
compare_errors_abs = []
for name in [('v1_0', 'v1_shift'), ('i12_0', 'i12_shift')]:
    compare_errors_abs.append(np.absolute(np.sqrt(3/2)*ts_dpsim_EMT[name[0]].values - ts_dpsim_DP_shifted[name[1]].values).max())
    print(name[0]+' vs. '+name[1] + ' (abs): ' + str(compare_errors_abs[-1]))

assert compare_errors_abs[0] < 1e-3 # voltage
assert compare_errors_abs[1] < 0.9  # current

### Assertion DP vs. SP

In [ ]:
compare_errors_abs = []
for name in ['v1', 'i12']:
    compare_errors_abs.append(np.absolute(ts_dpsim_DP[name].values - ts_dpsim_SP[name].values).max())
    print(name+' vs. '+name + ' (abs): ' + str(compare_errors_abs[-1]))

assert compare_errors_abs[0] < 1e-3 # voltage
assert compare_errors_abs[1] < 0.2    # current